# BEE 4750 Lab 3: Linear Programming with JuMP

**Name**:

**ID**:

> **Due Date**
>
> Wednesday, 10/16/24, 9:00pm

## Setup

The following code should go at the top of most Julia scripts; it will
load the local package environment and install any needed packages. You
will see this often and shouldn’t need to touch it.

In [49]:
import Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `c:\Users\arisc\OneDrive\Desktop\BEE 4750\lab-3-arischor-Fall24`


In [50]:
using JuMP # optimization modeling syntax
using HiGHS # optimization solver

## Overview

In this lab, you will write and solve a resource allocation example
using `JuMP.jl`. `JuMP.jl` provides an intuitive syntax for writing,
solving, and querying optimization problems.

`JuMP` requires the loading of a solver. \[Each supported solver works
for certain classes of problems, and some are open source while others
require a commercial license\]. We will use the `HiGHS` solver, which is
open source and works for linear, mixed integer linear, and quadratic
programs.

In this lab we will walk through the steps involved in coding a linear
program in HiGHS, solving it, and querying the solution.

## Exercise (3 points)

Your task is to decide how much lumber to produce to maximize profit
from wood sales. You can purchase wood from a managed forest, which
consists of spruce (320,000 bf) and fir (720,000 bf). Spruce costs
$\$0.12$ per bf to purchase and fir costs $\$0.08$ per bf.

At the lumber mill, wood can be turned into plywood of various grades
(see <a href="#tbl-inputs" class="quarto-xref">Table 1</a> for how much
wood of each type is required for and the revenue from each grade). Any
excess wood is sent to be recycled into particle board, which yields no
revenue for the mill.

| Plywood Grade | Inputs (bf/bf plywood) | Revenue (\$/1000 bf) |
|:-------------:|:----------------------:|:--------------------:|
|       1       |   0.5 (S) + 1.5 (F)    |         400          |
|       2       |   1.0 (S) + 2.0 (F)    |         520          |
|       3       |   1.5 (S) + 2.0 (F)    |         700          |

Table 1: Wood inputs and revenue by plywood grade. S refers to spruce
inputs, F fir inputs.

First, we need to identify our decision variables. While there are
several options, we will use $G_i$, the amount of each grade the mill
produces (in \$/1000 bf).

Using these decision variables, formulate a linear program to maximize
the profit of the mill subject to the supply constraints on spruce and
fir.

> **JuMP Syntax**
>
> The core pieces of setting up a `JuMP` model involve specifying the
> model and adding variables, the objective, and constraints. At the
> most simple level, this syntax looks like this:
>
> ``` julia
> m = Model(HiGHS.Optimizer)
> @variable(m, lb <= x <= ub) # if you do not have upper or lower bounds, you can drop those accordingly
> @variable(m, lb <= y <= ub)
> @objective(m, Max, 100x + 250y) # replace Max with Min depending on the problem
> @constraint(m, label, 6x + 12y <= 80) # replace "label" with some meaningful string you would like to use later to query shadow prices, or drop it
> ```
>
> You can add more constraints or more variables as needed.

> **Using Array Syntax**
>
> You can set up multiple variables or constraints at once using array
> syntax. For example, the following are equivalent:
>
> ``` julia
> @variable(m, G1 >= 0)
> @variable(m, G2 >= 0)
> @variable(m, G3 >= 0)
> ```
>
> and
>
> ``` julia
> @variable(m, G[1:3] >= 0)
> ```
>
> You can also set up multiple constraints using arrays of coefficients
> and/or bounds. For example:
>
> ``` julia
> I = 1:3
> d = [0; 3; 5]
> @constraint(m, demand[i in I], G[i] >= d[i])
> ```

`JuMP` is finicky about changing objects and constraints, so I recommend
setting up all of the model syntax in one notebook cell, which is what
we will do here.

In [51]:
forest_model = Model(HiGHS.Optimizer) # initialize model object
@variable(forest_model, G[1:3] >= 0) # non-negativity constraints
# uncomment the following lines and add the objective and constraints as needed for the model
 @objective(forest_model, Max, 220*G[1] + 240*G[2] + 360*G[3])
 @constraint(forest_model, fir, 0.5*G[1]+1.0*G[2]+1.5*G[3] <= 320)
 @constraint(forest_model, spruce, 1.5*G[1]+2.0*G[2]+2.0*G[3] <= 720)
print(forest_model) # this outputs a nicely formatted summary of the model so you can check your specification

Max 220 G[1] + 240 G[2] + 360 G[3]
Subject to
 fir : 0.5 G[1] + G[2] + 1.5 G[3] <= 320
 spruce : 1.5 G[1] + 2 G[2] + 2 G[3] <= 720
 G[1] >= 0
 G[2] >= 0
 G[3] >= 0


Next, to optimize, use the `optimize!()` function:

In [52]:
optimize!(forest_model)

Running HiGHS 1.7.2 (git hash: 5ce7a2753): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [5e-01, 2e+00]
  Cost   [2e+02, 4e+02]
  Bound  [0e+00, 0e+00]
  RHS    [3e+02, 7e+02]
Presolving model
2 rows, 3 cols, 6 nonzeros  0s
2 rows, 3 cols, 6 nonzeros  0s
Presolve : Reductions: rows 2(-0); columns 3(-0); elements 6(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -8.1999929744e+02 Ph1: 2(8.5); Du: 3(819.999) 0s
          2     1.1200000000e+05 Pr: 0(0) 0s
Model   status      : Optimal
Simplex   iterations: 2
Objective value     :  1.1200000000e+05
HiGHS run time      :          0.00


You should get confirmation that a solution was found; if one was not,
there’s a chance something was wrong with your model formulation.

To find the values of the decision variables, use `value()` (which can
be broadcasted over variable arrays):

In [53]:
@show value.(G);

value.(G) = [352.0, 0.0, 96.0]


Similarly, `objective_value()` finds the optimal value of the objective:

In [54]:
@show objective_value(forest_model);

objective_value(forest_model) = 112000.0


Finally, we can find the dual values of the constraints with
`shadow_price()`. Do this for the constraints in your model using the
block below.

In [55]:
 @show shadow_price(fir);
 @show shadow_price(spruce);

shadow_price(fir) = 80.0
shadow_price(spruce) = 120.0


`JuMP` also lets you evaluate other expressions that you might be
interested in based on the solutions. For example, you can use the
following block to calculate the total amount of plywood the mill would
produce under the optimal solution:

In [56]:
@expression(forest_model, total_plywood, sum(G))
@show value.(total_plywood);

value.(total_plywood) = 448.0


### Writeup

In this lab, we had to find how much of each grade of plywood to produce to maximize profits. Since the objective was maximizing profit, but we were only given revenue and prices, we had to first calculate the profit from each grade of plywood. $Profit = Revenue - Cost$, so to find the profit from each grade, we had to use the ratios and revenues in the table. In general, we would use the equation ${Profit}_{Grade} = {Rev}_{Grade} - ({Price}_{Spruce}*{Fraction}_{Spruce}*Grade + {Price}_{Fir}*{Fraction}_{Fir}*Grade)$. We also need to make sure that all of our units match up. Since we initialized the amount in each grade in $1000 bf$, we need to convert the prices to ${Price}_{Spruce} = 120\frac{dollars}{1000 bf}$ and ${Price}_{Fir} = 120\frac{dollars}{1000 bf}$. Then, we can calculate that ${Profit}_{G_1} = G_1*(400 - 120*0.5 - 80*1.5)$, ${Profit}_{G_2} = G_2*(520 - 120 - 80*2)$, and ${Profit}_{G_3} = G_3*(700 - 120*1.5 - 80*2)$. Then, we need to input our restraints, which is that we can't have negative amounts of any grade or type of wood and that we can't use more wood of either type than is in the forest. The nonnegativity constraint is simple, it is just that $G_1, G_2, G_3 \geq 0$. For the wood amount constraints, we have to use the ratios from the table to get the total amount of each wood: $0.5*G_1 + 1.0*G_2 + 1.5*G_3 \leq 320$ and $1.5*G_1 + 2.0*G_2 + 2.0*G_3$. We have to say that the constraint is 320 and 720 for each because G is in $1000 bf$ so we have to convert units. Finally we run it through the model to calculate that the optimal value is $G_1 = 352,000 bf$, $G_2 = 0 bf$, and $G_3 = 96,000 bf$.

## References

Put any consulted sources here, including classmates you worked with/who
helped you.

Sat next to and worked together with Jiaming Yuan and Grace Raab.